In [ ]:
!pip install understat

In [ ]:
import asyncio
import json
import pandas as pd

import aiohttp
from understat import Understat

async def fetch_data(session, team_title, season):
    understat = Understat(session)
    players = await understat.get_league_players(
        league_name="epl",
        season=season,
        team_title=team_title
    )
    df = pd.DataFrame(players)
    return df

async def main():
    # List of teams and seasons
    teams_seasons = [('Manchester City', 2022), ('Manchester City', 2023), ('Liverpool', 2022), ('Liverpool', 2023),
                     ('Manchester United', 2022), ('Manchester United', 2023), ('Arsenal', 2022), ('Arsenal', 2023),
                     ('Tottenham', 2022), ('Tottenham', 2023), ('Aston Villa', 2022), ('Aston Villa', 2023),
                     ('Everton', 2022), ('Everton', 2023)]

    async with aiohttp.ClientSession() as session:
        tasks = []
        for team, season in teams_seasons:
            tasks.append(fetch_data(session, team, season))

        # Gather all tasks
        dfs = await asyncio.gather(*tasks)

        # Merge DataFrames
        merged_df = pd.concat(dfs, ignore_index=True)

        # Save merged DataFrame to CSV
        merged_df.to_csv('understat_data_merged.csv', index=False)

# Run the asyncio event loop
await main()
